In [1]:
#!ls ~/trained_models/

In [5]:
from itertools import chain
from pathlib import Path
from datatype_recovery.models.dataset import load_dataset_from_path, max_typesequence_len_in_dataset
from datatype_recovery.models.dataset.encoding import *

import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from tqdm import tqdm

from datatype_recovery.models.metrics import acc_heuristic_numcorrect

def eval_model_on_dataset(model_path:Path, device:str, dataset_path:Path) -> float:
    '''
    Evaluates the model on the given dataset and returns the accuracy of the corrected
    model output against the dataset labels
    '''
    dataset = load_dataset_from_path(dataset_path)
    max_true_seq_len = max_typesequence_len_in_dataset(dataset_path)
    return eval_model_on_subset(model_path, device, dataset, max_true_seq_len)

def eval_model_on_subset(model_path:Path, device:str, dataset, max_true_seq_len:int) -> float:
    '''
    Evaluates the model on the given subset and returns the accuracy of the corrected
    model output against the dataset labels
    '''
    model = torch.load(model_path)
    print(model)

    # take the max of model seq length and max seq length of dataset so we
    # calculate accuracy correctly (without truncating something)
    max_len = max(model.max_seq_len, max_true_seq_len)

    # prepare the data loaders
    batch_size = 64
    dataset.transform = T.Compose([ToBatchTensors(), ToFixedLengthTypeSeq(max_len)])

    # split the dataset into the part divisible by batch size and the leftovers
    # we can chain these together for performance - our metrics simply iterate
    # through all elements in the batch
    batched_total = len(dataset)-(len(dataset)%batch_size)
    batch_loader = DataLoader(dataset[:batched_total], batch_size=batch_size)
    leftovers_loader = DataLoader(dataset[batched_total:], batch_size=1)

    print(f'Running eval...')

    model.to(device)
    model.eval()
    num_correct = 0

    for data in tqdm(chain(batch_loader, leftovers_loader), total=len(batch_loader)+len(leftovers_loader)):
        data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        num_correct += acc_heuristic_numcorrect(data.y, out)

    accuracy = num_correct/len(dataset)
    print(f'Accuracy = {accuracy*100:,.2f}%')

    return accuracy

In [ ]:
# TODO: eval on an interesting subset of the data
# from torch.utils.data import Subset
# Subset()

In [6]:
dataset_path = Path.home()/'trainset_astera_5hops_nocomp'
model_path = Path.home()/'trained_models/structural_3out_3hops_nocomp.pt'

eval_model_on_dataset(model_path, 'cuda:3', dataset_path)

Calculating max true sequence length in dataset trainset_astera_5hops_nocomp


100%|██████████| 24691/24691 [00:19<00:00, 1253.10it/s]


StructuralTypeSeqModel(
  (gat_layers): ModuleList(
    (0): GATConv(31, 128, heads=1)
    (1-2): 2 x GATConv(128, 128, heads=1)
  )
  (pred_head): Linear(128, 66, bias=True)
)
Running eval...


100%|██████████| 436/436 [00:17<00:00, 24.34it/s]


Accuracy = 2.22%


0.022234822404924873

In [7]:
dataset = load_dataset_from_path(dataset_path)

In [9]:
dataset[0].varid

(0, 0, 1323424, '17', 'l')

In [22]:
import pandas as pd

# pd.read_csv(dataset.root/dataset.raw_file_names[0])
from datatype_recovery.models.dataset import TypeSequenceDataset

ds = TypeSequenceDataset(dataset_path)

In [34]:
exp_runs = pd.read_csv(ds.exp_runs_path)
exp_runs

,RunGid,RunFolder,BinariesCsv,FuncsCsv,ParamsCsv,LocalsCsv
0,0,/home/cls0027/exp_builds/astera.exp/rundata/run1,/home/cls0027/exp_builds/astera.exp/rundata/ru...,/home/cls0027/exp_builds/astera.exp/rundata/ru...,/home/cls0027/exp_builds/astera.exp/rundata/ru...,/home/cls0027/exp_builds/astera.exp/rundata/ru...


In [ ]:

# TODO: to slice and dice this dataset, we really need to UNIFY the data frames (binaries/funcs/params/locals)
# (NOTE this should actually happen during initial dataset creation, unless we specify a --quick mode or something)
#
# we HAVE RunGid mapping (RunGID->files)
#
# --> convert (local) binary IDs to a global binary ID (binary GID)
# (find # binaries in each run, take the max # binaries and round up to the NEXT largest 1,000...this is the BASE binary GID for this run)
# (now add each binary ID to the BASE to get its GID, for all tables (bins/funcs/params/locals))
# (save the base in this exp_runs table)
#       --> HAVE TO COPY THE FILES LOCALLY during this step so we don't overwrite the originals!
# --> pd.concat() all tables across all runs for each table type, one at a time (binary GIDs should now be unique)

# ...NOW we should be able to run our analysis to slice/dice the data based on the
# whole dataset (split/index using the varid's for individual data points)